In [1]:
##################################################################
#           《Python人工智能编程实践（2024年度版）》开源代码
#-----------------------------------------------------------------
#            @章节号：6.3.1（卷积神经网络的PyTorch实践）                                                    
#            @作者：范淼、徐晟桐 
#            @购书链接：暂无
#            @电子邮箱：fm12@tsinghua.org.cn             
#            @官方交流QQ群号：561500762                        
##################################################################

In [2]:
from torch import nn, optim


#设定超参数。
FILTERS = (32, 64)
HIDDEN_SIZE = 256
NUM_CLASSES = 10
EPOCHS = 5
BATCH_SIZE = 64
LEARNING_RATE = 1e-3


class CNN(nn.Module):
    '''
    自定义卷积神经网络类，继承自nn.Module。
    '''
    def __init__(self, filters, hidden_size, num_classes):
        
        super(CNN, self).__init__() 
        
        self.conv_1 = nn.Conv2d(in_channels=1, out_channels=filters[0], kernel_size=3, padding='same')
        
        self.conv_2 = nn.Conv2d(in_channels=filters[0], out_channels=filters[1], kernel_size=3, padding='same')
        
        self.relu = nn.ReLU()
        
        self.max_pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.flatten = nn.Flatten()
        
        self.l1 = nn.Linear(int (28 / 2) ** 2 * filters[1], hidden_size)
        
        self.l2 = nn.Linear(hidden_size, num_classes)

        
    def forward(self, input_tensor):

        out = self.conv_1(input_tensor)
        out = self.relu(out)
        
        out = self.conv_2(out)
        out = self.relu(out)
        
        out = self.max_pool(out)
        
        out = self.flatten(out)
        
        out = self.l1(out)
        out = self.relu(out)
        
        out = self.l2(out)
        
        return out 
    

#初始化卷积神经网络。
model = CNN(FILTERS, HIDDEN_SIZE, NUM_CLASSES)

#设定神经网络的损失函数。
criterion = nn.CrossEntropyLoss()

#设定神经网络的优化方法。
optimizer = optim.Adam(model.parameters(), lr = LEARNING_RATE) 

In [3]:
import pandas as pd


#使用pandas，读取fashion_mnist的训练和测试数据文件。
train_data = pd.read_csv('./datasets/fashion_mnist/fashion_mnist_train.csv')
test_data = pd.read_csv('./datasets/fashion_mnist/fashion_mnist_test.csv')

#从训练数据中，拆解出训练特征和类别标签。
X_train = train_data[train_data.columns[1:]]
y_train = train_data['label']

#从测试数据中，拆解出测试特征和类别标签。
X_test = test_data[train_data.columns[1:]]
y_test = test_data['label']

In [4]:
from sklearn.preprocessing import StandardScaler


#初始化数据标准化处理器。
ss = StandardScaler()

#标准化训练数据特征。
X_train = ss.fit_transform(X_train)

#标准化测试数据特征。
X_test = ss.transform(X_test)

In [5]:
import torch
from torch.utils.data import TensorDataset, DataLoader


#构建适用于PyTorch模型训练的数据结构。
train_tensor = TensorDataset(torch.tensor(X_train.astype('float32')), torch.tensor(y_train.values))
  
#构建适用于PyTorch模型训练的数据读取器。                       
train_loader = DataLoader(dataset = train_tensor, batch_size = BATCH_SIZE, shuffle = True)

n_total_steps = len(train_loader)

#开启模型训练。
model.train()

for epoch in range(EPOCHS):
    for i, (features, labels) in enumerate(train_loader): 
        images = features.reshape([-1, 1, 28, 28])
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step() 
        
        if (i+1) % 300 == 0:
             print (f'Epoch [{epoch+1}/{EPOCHS}], Step[{i+1}/{n_total_steps}], Loss: {loss.item():.4f}') 

Epoch [1/5], Step[300/938], Loss: 0.5093
Epoch [1/5], Step[600/938], Loss: 0.3102
Epoch [1/5], Step[900/938], Loss: 0.1799
Epoch [2/5], Step[300/938], Loss: 0.2389
Epoch [2/5], Step[600/938], Loss: 0.1263
Epoch [2/5], Step[900/938], Loss: 0.1540
Epoch [3/5], Step[300/938], Loss: 0.0510
Epoch [3/5], Step[600/938], Loss: 0.0953
Epoch [3/5], Step[900/938], Loss: 0.1696
Epoch [4/5], Step[300/938], Loss: 0.2009
Epoch [4/5], Step[600/938], Loss: 0.0372
Epoch [4/5], Step[900/938], Loss: 0.1303
Epoch [5/5], Step[300/938], Loss: 0.0822
Epoch [5/5], Step[600/938], Loss: 0.0121
Epoch [5/5], Step[900/938], Loss: 0.0394


In [6]:
#构建适用于PyTorch模型测试的数据结构。
test_tensor = TensorDataset(torch.tensor(X_test.astype('float32')), torch.tensor(y_test.values))

#构建适用于PyTorch模型测试的数据读取器。
test_loader = DataLoader(dataset = test_tensor, batch_size = BATCH_SIZE, shuffle = False)

#开启模型测试。
model.eval()

n_correct = 0
n_samples = 0

for features, labels in test_loader:
    images = features.reshape([-1, 1, 28, 28])
    outputs = model(images)
    _, predictions = torch.max(outputs.data, 1)
    
    n_samples += labels.size(0)
    n_correct += (predictions == labels).sum().item() 

acc = 100.0 * n_correct / n_samples

print('卷积神经网络（PyTorch版本）在fashion_mnist测试集上的准确率为: %.2f%%。' %acc) 

卷积神经网络（PyTorch版本）在fashion_mnist测试集上的准确率为: 92.64%。
